In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pathlib import Path
import os
import pandas as pd
import scanpy as sc
import cellink as cl

/opt/modules/i12g/anaconda/envs/mzb_scgenetics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from cellink.tl._annotate_snps_genotype_data import *

In [5]:
from cellink.tl._burden_testing import *

In [6]:
# Paths
base_data_dir = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/")
scdata_path = base_data_dir / "input_data/OneK1K_cohort_gene_expression_matrix_14_celltypes_w_gene_locations.h5ad.gz"
gdata_dir = Path("/data/ceph/hdd/project/node_09/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data/filter_vcf_r08/")

# READ FILES
zarr_file = gdata_dir / "chr22.dose.filtered.R2_0.8.vcz"
#zarr_file = os.path.join(gdata_dir, f"chr{str(args.chromosome)}.dose.filtered.R2_0.8.vcz")
eigenvec = pd.read_csv(base_data_dir / "input_data/pcdir/wgs.dose.filtered.R2_0.8.filtered.pruned.eigenvec", sep = ' ')
DNA_LM_upstream = base_data_dir/ "input_data/annotations/onek1k_inf_scores_upstream_model.tsv"
DNA_LM_downstream = base_data_dir/ "input_data/annotations/onek1k_inf_scores_downstream_model.tsv"
vep_scores = base_data_dir/ "input_data/annotations/onek1k1_all_variants_annotated_vep.txt"


In [15]:
scdata = sc.read_h5ad(scdata_path)
gdata = cl.io.read_sgkit_zarr(zarr_file)

In [7]:
from cellink.tl.utils import (
    _add_dummy_cols,
    _explode_columns,
    _get_vep_start_row,
)

annos = pd.read_csv(vep_scores, sep="\t", skiprows=_get_vep_start_row(vep_scores))
annos

/scratch/tmp/l_back/ipykernel_1952352/1980214688.py:7: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  annos = pd.read_csv(vep_scores, sep="\t", skiprows=_get_vep_start_row(vep_scores))


#Uploaded_variation     Location Allele             Gene  \
0               1_715265_C/T     1:715265      T  ENSG00000237491   
1               1_715265_C/T     1:715265      T  ENSG00000228327   
2               1_715367_A/G     1:715367      G  ENSG00000228327   
3               1_715367_A/G     1:715367      G  ENSG00000237491   
4               1_717485_C/A     1:717485      A  ENSG00000228327   
...                      ...          ...    ...              ...   
14044678     9_141072019_G/A  9:141072019      A  ENSG00000159247   
14044679     9_141073623_G/A  9:141073623      A  ENSG00000159247   
14044680     9_141084200_G/A  9:141084200      A                -   
14044681     9_141089144_C/T  9:141089144      T  ENSG00000237419   
14044682     9_141101939_C/T  9:141101939      T  ENSG00000233013   

                  Feature Feature_type  \
0         ENST00000434264   Transcript   
1         ENST00000428504   Transcript   
2         ENST00000428504   Transcript   
3         ENST00000434264   Transcript   
4         ENST00000428504   Transcript   
...                   ...          ...   
14044678  ENST00000508529   Transcript   
14044679  ENST00000508529   Transcript   
14044680                -            -   
14044681  ENST00000428088   Transcript   
14044682  ENST00000540522   Transcript   

                                           Consequence cDNA_position  \
0         intron_variant,non_coding_transcript_variant             -   
1                                upstream_gene_variant             -   
2                                upstream_gene_variant             -   
3         intron_variant,non_coding_transcript_variant             -   
4                                upstream_gene_variant             -   
...                                                ...           ...   
14044678                       downstream_gene_variant             -   
14044679                       downstream_gene_variant             -   
14044680                            intergenic_variant             -   
14044681                       downstream_gene_variant             -   
14044682                         upstream_gene_variant             -   

         CDS_position Protein_position  ... gnomADe_FIN_AF gnomADe_NFE_AF  \
0                   -                -  ...              -              -   
1                   -                -  ...              -              -   
2                   -                -  ...              -              -   
3                   -                -  ...              -              -   
4                   -                -  ...              -              -   
...               ...              ...  ...            ...            ...   
14044678            -                -  ...              -              -   
14044679            -                -  ...              -              -   
14044680            -                -  ...              -              -   
14044681            -                -  ...              -              -   
14044682            -                -  ...              -              -   

         gnomADe_OTH_AF gnomADe_SAS_AF CLIN_SIG SOMATIC PHENO CADD_PHRED  \
0                     -              -        -       -     -      1.266   
1                     -              -        -       -     -      1.266   
2                     -              -        -       -     -      2.493   
3                     -              -        -       -     -      2.493   
4                     -              -        -       -     -      0.400   
...                 ...            ...      ...     ...   ...        ...   
14044678              -              -        -       -     -      6.198   
14044679              -              -        -       -     -      3.765   
14044680              -              -        -       -     -      7.328   
14044681              -              -        -       -     -      0.029   
14044682              -              -        -  

In [10]:
filtered_annos = annos[annos["Location"].str.split(":").str[0] == "22"]

In [29]:
filtered_annos.loc[7828337, "DISTANCE"]

'-'

In [12]:
filtered_annos.to_csv("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data/annotations/onek1k1_chr22_variants_annotated_vep.txt", sep="\t", index=False)

In [48]:
vep_chr22="/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data/annotations/onek1k1_chr22_variants_annotated_vep.txt"
cl.tl.add_vep_annos_to_gdata(vep_chr22, gdata,
                             cols_to_explode=["Consequence"],
                             cols_to_dummy=["Consequence"])

[2025-01-18 17:02:33,450] INFO:cellink.tl._annotate_snps_genotype_data: renaming id column #Uploaded_variation into snp_id
[2025-01-18 17:02:33,584] INFO:cellink.tl._annotate_snps_genotype_data: Subsetting annotations to variants that are in gdata
[2025-01-18 17:02:33,585] INFO:cellink.tl._annotate_snps_genotype_data: 0 with missing annotation
[2025-01-18 17:02:34,037] INFO:cellink.tl._annotate_snps_genotype_data: Index(['snp_id', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type',
       'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position',
       'Amino_acids', 'Codons', 'Existing_variation', 'IMPACT', 'DISTANCE',
       'STRAND', 'FLAGS', 'BIOTYPE', 'CANONICAL', 'ENSP', 'SIFT', 'PolyPhen',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'CADD_PHRED',
       'CADD_RAW', 'TSSDistance'],
      dtype='object

AnnData object with n_obs × n_vars = 1034 × 143083
    var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2', 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'
    varm: 'annotations_0', 'annotations_1', 'annotations_2', 'annotations_3', 'annotations_4', 'annotations_5', 'annotations_6', 'annotations_7'

In [20]:
type(scdata.var.loc['ENSG00000243485', "start"])#[gene]

numpy.float64

In [17]:
scdata.var

GeneSymbol       features chromosome    start      end
Geneid                                                                    
ENSG00000243485     MIR1302-10     MIR1302-10          1  28589.0  31109.0
ENSG00000237613        FAM138A        FAM138A          1  37595.0  34553.0
ENSG00000186092          OR4F5          OR4F5          1  65419.0  71585.0
ENSG00000238009   RP11-34P13.7   RP11-34P13.7        NaN      NaN      NaN
ENSG00000239945   RP11-34P13.8   RP11-34P13.8          1  91105.0  89551.0
...                        ...            ...        ...      ...      ...
ENSG00000215635     AC145205.1     AC145205.1        NaN      NaN      NaN
ENSG00000268590          BAGE5          BAGE5        NaN      NaN      NaN
ENSG00000251180     CU459201.1     CU459201.1        NaN      NaN      NaN
ENSG00000215616     AC002321.2     AC002321.2        NaN      NaN      NaN
ENSG00000215611     AC002321.1     AC002321.1        NaN      NaN      NaN

[32738 rows x 5 columns]

In [30]:
all_res_chr22=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/chr22_all_results_DNA_LM_down_and_MAF_100k.pkl")

In [33]:
all_res_chr22[(all_res_chr22["CADD_PHRED"]!=0) & (~np.isnan(all_res_chr22["CADD_PHRED"]))]

DISTANCE  CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  \
id                                                                        
1_1        1377789.0    9350.801  69.814536   133.275103   43612.917035   
2_2        1395621.0   10775.218  77.605564   151.365002   52127.930292   
3_3        1319479.0    8327.426  62.418011   119.240087   39069.213569   
4_4        1434398.0    9439.675  65.065089   129.037760   44983.557785   
6_6        1428912.0    9054.896  66.864204   127.860172   42525.702648   
...              ...         ...        ...          ...            ...   
1077_1078  1422923.0    9165.348  63.239808   124.158291   42490.914665   
1078_1079  1366478.0    9077.036  66.221163   129.565329   43114.049369   
1079_1080  1595183.0    9811.044  72.452906   139.280093   43941.663597   
1080_1081  1323326.0    8346.966  61.066604   119.626789   40767.972544   
1081_1082  1564430.0    8972.718  65.330756   128.064908   42813.245027   

                    Geneid  
id                          
1_1        ENSG00000233866  
2_2        ENSG00000233866  
3_3        ENSG00000233866  
4_4        ENSG00000233866  
6_6        ENSG00000233866  
...                    ...  
1077_1078  ENSG00000079974  
1078_1079  ENSG00000079974  
1079_1080  ENSG00000079974  
1080_1081  ENSG00000079974  
1081_1082  ENSG00000079974  

[642555 rows x 6 columns]

In [46]:
gdata[:, ["22_16849573_A_G","22_16849971_A_T"]].varm["filter"]
#this_weights = np.array(gdata[:, ["22_16849573_A_G","22_16849971_A_T"]].varm['annotations_0'][weight_col])

PASS  GENOTYPED  GENOTYPED_ONLY
snp_id                                          
22_16849573_A_G  True      False           False
22_16849971_A_T  True      False           False

In [55]:
all_res_chr22

DISTANCE  CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  \
id                                                                       
1_1             0.0         0.0        0.0          0.0            0.0   
2_2             0.0         0.0        0.0          0.0            0.0   
3_3             0.0         0.0        0.0          0.0            0.0   
4_4             0.0         0.0        0.0          0.0            0.0   
6_6             0.0         0.0        0.0          0.0            0.0   
...             ...         ...        ...          ...            ...   
1077_1078       NaN         NaN        NaN          NaN            NaN   
1078_1079       NaN         NaN        NaN          NaN            NaN   
1079_1080       NaN         NaN        NaN          NaN            NaN   
1080_1081       NaN         NaN        NaN          NaN            NaN   
1081_1082       NaN         NaN        NaN          NaN            NaN   

                    Geneid  
id                          
1_1        ENSG00000243485  
2_2        ENSG00000243485  
3_3        ENSG00000243485  
4_4        ENSG00000243485  
6_6        ENSG00000243485  
...                    ...  
1077_1078  ENSG00000215611  
1078_1079  ENSG00000215611  
1079_1080  ENSG00000215611  
1080_1081  ENSG00000215611  
1081_1082  ENSG00000215611  

[32115978 rows x 6 columns]